In [78]:
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression
import numpy as np
import datetime as dt

In [79]:
ticker = 'AAPL'
API = 'ec2d6f3b0cc9395fbce02f1316669229'
url = (f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?apikey={API}")

r = requests.get(url)
cash_flow_statement = r.json()

In [80]:
url1 = (F'https://financialmodelingprep.com/api/v3/quote-short/{ticker}?apikey={API}')

f = requests.get(url1)
quote = f.json()

df_quote = pd.DataFrame(quote)
price = df_quote['price'].iloc[0]  # This extracts the first value in the 'price' Series as a scalar
print(f'Price: {price}')

Price: 188.791


In [81]:
df_cash_flow = pd.DataFrame(cash_flow_statement)


In [82]:
df_cash_flow.head()

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,netIncome,depreciationAndAmortization,...,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink
0,2023-09-30,AAPL,USD,0000320193,2023-11-03,2023-11-02 18:08:27,2023,FY,96995000000,11519000000,...,-108488000000,0,5760000000,30737000000,24977000000,110543000000,-10959000000,99584000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
1,2022-09-24,AAPL,USD,0000320193,2022-10-28,2022-10-27 18:01:14,2022,FY,99803000000,11104000000,...,-110749000000,0,-10952000000,24977000000,35929000000,122151000000,-10708000000,111443000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
2,2021-09-25,AAPL,USD,0000320193,2021-10-29,2021-10-28 18:04:28,2021,FY,94680000000,11284000000,...,-93353000000,0,-3860000000,35929000000,39789000000,104038000000,-11085000000,92953000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
3,2020-09-26,AAPL,USD,0000320193,2020-10-30,2020-10-29 18:06:25,2020,FY,57411000000,11056000000,...,-86820000000,0,-10435000000,39789000000,50224000000,80674000000,-7309000000,73365000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...
4,2019-09-28,AAPL,USD,0000320193,2019-10-31,2019-10-30 18:12:36,2019,FY,55256000000,12547000000,...,-90976000000,0,24311000000,50224000000,25913000000,69391000000,-10495000000,58896000000,https://www.sec.gov/Archives/edgar/data/320193...,https://www.sec.gov/Archives/edgar/data/320193...


## Operating Cash Flows

In [83]:
for OFC in df_cash_flow['operatingCashFlow']:
    print(OFC)

110543000000
122151000000
104038000000
80674000000
69391000000


In [84]:
# FCF Cash Flow available to both debt an equity holders after the business pays for everything it needs to continue operating
for FCF in df_cash_flow['freeCashFlow']:
    print(FCF)

99584000000
111443000000
92953000000
73365000000
58896000000


## DCF
A DCF is a valuation methodology that measures the intrinsic value of a company based on the sum of the present value of its future cash flows --> More simple answer: a DCF measures the value of an asset based on the amount of cash it can produce

1. Forecast the free cash flow --> The first step is to project out for 5-10 years by making assumptions on the company's revenue growth and EBIT margins. (in my case the assumptions where made using the scikit learn package)

2. Calculated the weighted Average cost of capital (WACC) --> Rate of return required by debt equity investors for a company to fund the growth of its future free cash flow.

    present value of your FCF --> WACC = (% of equity * cost of equity) + (% of Debt * cost of Debt * (1-Tax Rate))

    To calculate the cost of Equity I used the following formula: CAPM = Capital Asset Pricing Model: Risk Free rate + (beta * expected Market return - risk free rate).
    For equity value: Market cap --> debt + equity value(market cap)
    For percentage of debt = debt / (debt + equity)
    For percentage of equity = equity / (debt + equity)

    The Risk free rate = 4.64%. It was obtained form: https://www.bloomberg.com/markets/rates-bonds/government-bonds/us I used the yield on a 5-year on a treasury bond.

    The beta for Apple = 3.1 for 5 yield period. It was obtained form: https://finance.yahoo.com/quote/AAPL/

    For the Market Return I used an average of 7.58% and 10.51% it is = to 9.05% according to: https://www.forbes.com/advisor/investing/average-stock-market-return/

    For the tax rate I will use 15.65% https://valueinvesting.io/AAPL/valuation/wacc

    For the cost of debt I'm using the following formula: Interest expense / total_debt * (1-tax rate)

## WACC

* Different industries have different average WACCs due to varying risk.
* A company with volatile cash flows or in volatile industry might have a higher WACC
* A company that is financed more by debt might have a higher WACC if the cost of debt is high, however debt is typically cheaper than equity due tax shields
* During Periods of high-interest rates, WACC tends to increase because the cost of borrowing goes up



3. Calculate the terminal value
## Terminal Growth Value
* The terminal growth rate is the steady rate at which a company's projected free cash flows are expected to increase indefinitely.
* For the purposes of this analysis, we will employ the average GDP growth rate as a proxy for the terminal growth rate.
* TGR = 2.1%


4. Discount the free cash flow and terminal value
5. Calculate the implied share price

In [85]:
#Forecaste Free Cash flows

#Sort Data Frame by date
df_cash_flow['date'] = pd.to_datetime(df_cash_flow['date'])
df_cash_flow = df_cash_flow.sort_values(by = 'date')

# Get only the year from the date
df_cash_flow['year'] = df_cash_flow['date'].dt.year

#Historical Data
years = df_cash_flow['year'].values.reshape(-1,1)
future_free_cash_flows = df_cash_flow['freeCashFlow'].values

ffcf = LinearRegression().fit(years, future_free_cash_flows)

future_years = np.array([2023, 2024, 2025, 2026, 2027]).reshape(-1,1)
predicted_ffcf = ffcf.predict(future_years)

predicted_cash_flows_list = []

for year, free_cash_flow in zip(future_years, predicted_ffcf):  # Removed the comma after 'free_cash_flow'
    print(f"Year: {year[0]}")
    print(f'Predicted Free Cash Flows: {free_cash_flow}')
    print("-------------------------")

    predicted_cash_flows_list.append(free_cash_flow)  # Append without print

# After the loop, you can print the entire list to confirm it has the values

# Forecasted the cash flows using scikit learn, however I'm going to calculate the cash flows manually.
# Free Cash Flow = EBIT * (1-tax rate) + Depreciation and Amortization --> ($no cash flow leaving the company$) - capital expenditures - increase in non-cash working capital
# Formula for the Non-Cash Working Capital = current Assets - cash - current liabilities

Year: 2023
Predicted Free Cash Flows: 111139000000.0
-------------------------
Year: 2024
Predicted Free Cash Flows: 123084400000.0
-------------------------
Year: 2025
Predicted Free Cash Flows: 135029800000.0
-------------------------
Year: 2026
Predicted Free Cash Flows: 146975200000.0
-------------------------
Year: 2027
Predicted Free Cash Flows: 158920600000.0
-------------------------


In [86]:
%%capture
%run Income_Statement.ipynb # get data from the income statement
%run Balance_Sheet.ipynb # get data from the balance sheet

In [87]:
# Calculating the WACC = (% of equity * cost of equity) + (% of Debt * cost of Debt * (1-Tax Rate))
# Cost of Equity = Risk Free rate + (beta * (expected Market return - risk free rate))
risk_free_rate = 0.0482
beta = 1.30
market_return = 0.0945
tax_rate = 0.1565

cost_of_equity = risk_free_rate + (beta * (market_return - risk_free_rate))
print(f'Cost of Equity: {cost_of_equity}')

sharesoutstanding = 15787154000
total_debt = df_balance_sheet['totalDebt'][0]
value_of_equity = price * sharesoutstanding

total =(value_of_equity + total_debt)


print('-----------------------')

percentage_debt = total_debt / total
print(f'Percentage of Debt:{percentage_debt}')

print('-----------------------')

percentage_equity = value_of_equity / total
print(f'Percentage of Equity:{percentage_equity}')

print('-----------------------')

cost_of_debt = interest_exp / total_debt * (1-tax_rate)
print(f'Cost of Debt: {cost_of_debt}')

Cost of Equity: 0.10839
-----------------------
Percentage of Debt:0.035932661429983755
-----------------------
Percentage of Equity:0.9640673385700163
-----------------------
Cost of Debt: 0.029863581124873972


In [88]:
# Calculating the WACC = (% of equity * cost of equity) + (% of Debt * cost of Debt * (1-Tax Rate))
wacc = (percentage_equity * cost_of_equity) + (percentage_debt * cost_of_debt * (1 - tax_rate))
print(f'WACC = {wacc:.2%}')

WACC = 10.54%


In [89]:
TGR = 0.021
def terminal_value(TGR, predicted_cash_flows_list, wacc):
    terminal_values = []

    for predicted_cash_flow in predicted_cash_flows_list:

        TV = predicted_cash_flow * (1 + TGR) / (wacc - TGR)
        terminal_values.append(TV)

    return terminal_values

terminal_values_by_year = []

TVs = terminal_value(TGR, predicted_cash_flows_list, wacc)
print("Terminal Values for each year's predicted cash flow:")
print()
for year, TV in enumerate(TVs, start=2024):
    formatted_tv = "{:,.2f}".format(TV)
    print(f"Year {year} Terminal Value: ${formatted_tv}")
    print('----------------------------')

    terminal_values_by_year.append(TV)

Terminal Values for each year's predicted cash flow:

Year 2024 Terminal Value: $1,344,459,491,838.38
----------------------------
Year 2025 Terminal Value: $1,488,964,178,886.18
----------------------------
Year 2026 Terminal Value: $1,633,468,865,933.99
----------------------------
Year 2027 Terminal Value: $1,777,973,552,981.79
----------------------------
Year 2028 Terminal Value: $1,922,478,240,029.60
----------------------------


In [90]:
def discount (predicted_cash_flows_list, wacc):
    discount_values  = []

    for i, predicted_cash_flow in enumerate(predicted_cash_flows_list, start=1):
        PV = predicted_cash_flow / (1 + wacc) ** i
        discount_values.append(PV)

    return discount_values

present_values_by_year = []

PVs = discount(predicted_cash_flows_list, wacc)
print('Discounted Values for each year')
print()
for year, PV in enumerate(PVs, start= 2024):
    formatted_pv = "{:,.2f}".format(PV)
    print(f'Year {year} Present Values: ${formatted_pv}')
    print('-------------------------')

    present_values_by_year.append(PV)

Discounted Values for each year

Year 2024 Present Values: $100,541,848,901.22
-------------------------
Year 2025 Present Values: $100,731,146,744.18
-------------------------
Year 2026 Present Values: $99,970,248,605.47
-------------------------
Year 2027 Present Values: $98,438,636,630.18
-------------------------
Year 2028 Present Values: $96,290,204,465.64
-------------------------


In [91]:
# Discount Factor 1/(1 + wacc) ** period

for i in range(1,6):
    discount_factors = 1/(1 + wacc) ** i
    print(discount_factors)

year2028 = float(terminal_values_by_year[-1])
pv_of_tv = 0.6058938531864974 * year2028

0.9046495730681106
0.8183908500523147
0.7403569331026745
0.6697635834493502
0.6059013398240225


In [92]:
# Sum of Pv of free cash flow
# + PV of terminal value
#------------------------------
# ENTERPRISE VALUE
# - net debt
# + preferred stock
# + non controlling interest
#-------------------------------
# EQUITY VALUE
# / Number of Shares
#--------------------------------
# IMPLIED SHARE PRICE
sum_of_pv = 0
for value in present_values_by_year:
    sum_of_pv += value
print()



enterprise_value = pv_of_tv + sum_of_pv
equity_value = enterprise_value - total_debt + cash


implied_share_price = equity_value / sharesoutstanding
print(f'Share Price: {implied_share_price}')


Share Price: 100.06026633207081
